In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# Cell 1: Train & save GAN generator
# ─────────────────────────────────────────────────────────────────────────────
import os
import pickle
import numpy as np

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import (
    Conv1D, BatchNormalization, LeakyReLU, Flatten, Dense,
    Embedding, Concatenate, UpSampling1D, Activation, Reshape, Dropout
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal

# 1) Load & prepare IQ data
with open("RML2016.10a_dict.pkl", "rb") as f:
    raw = pickle.load(f, encoding="latin1")

X, y, snr = [], [], []
for (mod, S), sigs in raw.items():
    for sig in sigs:
        X.append(np.vstack([sig[0], sig[1]]).T)  # (128,2)
        y.append(mod)
        snr.append(S)
X = np.array(X, dtype=np.float32)
y = np.array(y)
snr = np.array(snr)

# 2) Encode labels and split high‐SNR for GAN training
le = LabelEncoder(); y_enc = le.fit_transform(y)
mask = snr > 6
X_train = X[mask]; y_train = y_enc[mask]
n_classes = len(le.classes_)

# 3) GAN definitions
def define_discriminator(input_shape, n_classes):
    init = RandomNormal(stddev=0.02)
    inp = Input(shape=input_shape)
    x = Conv1D(32,3,padding='same',kernel_initializer=init)(inp)
    x = LeakyReLU(0.2)(x); x = Dropout(0.5)(x)
    x = Conv1D(64,3,strides=2,padding='same',kernel_initializer=init)(x)
    x = BatchNormalization()(x); x = LeakyReLU(0.2)(x); x = Dropout(0.5)(x)
    x = Conv1D(128,3,strides=2,padding='same',kernel_initializer=init)(x)
    x = BatchNormalization()(x); x = LeakyReLU(0.2)(x); x = Dropout(0.5)(x)
    x = Flatten()(x)
    out_real  = Dense(1, activation='sigmoid')(x)
    out_class = Dense(n_classes, activation='softmax')(x)
    model = Model(inp, [out_real, out_class])
    model.compile(optimizer=Adam(1e-4, beta_1=0.5),
                  loss=['binary_crossentropy','sparse_categorical_crossentropy'])
    return model

def define_generator(latent_dim, n_classes):
    init = RandomNormal(stddev=0.02)
    in_lbl   = Input(shape=(1,))
    li       = Embedding(n_classes, 50)(in_lbl)
    li       = Dense(latent_dim, kernel_initializer=init)(li)
    li       = Reshape((latent_dim,))(li)
    in_noise = Input(shape=(latent_dim,))
    merge    = Concatenate()([in_noise, li])
    x        = Dense(64*32, kernel_initializer=init)(merge)
    x        = Activation('relu')(x)
    x        = Reshape((32,64))(x)
    x        = UpSampling1D()(x)
    x        = Conv1D(64,5,padding='same',kernel_initializer=init)(x)
    x        = BatchNormalization()(x); x = Activation('relu')(x)
    x        = UpSampling1D()(x)
    x        = Conv1D(32,5,padding='same',kernel_initializer=init)(x)
    x        = BatchNormalization()(x); x = Activation('relu')(x)
    x        = Conv1D(2,5,padding='same',kernel_initializer=init)(x)
    out      = Activation('tanh')(x)
    return Model([in_noise, in_lbl], out)

def define_gan(gen, disc):
    for layer in disc.layers:
        if not isinstance(layer, BatchNormalization):
            layer.trainable = False
    out = disc(gen.output)
    m   = Model(gen.input, out)
    m.compile(optimizer=Adam(1e-4, beta_1=0.5),
              loss=['binary_crossentropy','sparse_categorical_crossentropy'])
    return m

# 4) Training utilities
def gen_latent(latent_dim, n, n_classes):
    return np.random.randn(n, latent_dim), np.random.randint(0, n_classes, n)

def gen_real(X, y, n):
    idx = np.random.randint(0, len(X), n)
    return [X[idx], y[idx]], np.ones((n,1))

def gen_fake(gen, latent_dim, n, n_classes):
    z, labels = gen_latent(latent_dim, n, n_classes)
    Xf = gen.predict([z, labels], verbose=0)
    return [Xf, labels], np.zeros((n,1))

# 5) Build & train GAN
latent_dim = 100; epochs_gan = 100; bs = 64
disc = define_discriminator((128,2), n_classes)
gen  = define_generator(latent_dim, n_classes)
gan  = define_gan(gen, disc)

half_bs = bs//2
steps   = max(1, len(X_train)//bs)
for _ in range(epochs_gan):
    for _ in range(steps):
        (Xr, yr), _   = gen_real(X_train, y_train, half_bs)
        disc.train_on_batch(Xr, [np.ones((half_bs,1)), yr])
        (Xf, yf), _   = gen_fake(gen, latent_dim, half_bs, n_classes)
        disc.train_on_batch(Xf, [np.zeros((half_bs,1)), yf])
        z, zl         = gen_latent(latent_dim, bs, n_classes)
        gan.train_on_batch([z, zl], [np.ones((bs,1)), zl])

# 6) Save generator for later
os.makedirs("saved_models", exist_ok=True)
gen.save("saved_models/gan_generator.keras")
print("✅ GAN generator saved to saved_models/gan_generator.keras")




2025-04-27 15:47:48.283303: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-27 15:47:48.295830: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745783268.308989 3125256 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745783268.312879 3125256 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-27 15:47:48.327323: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:

# ─────────────────────────────────────────────────────────────────────────────
# Cell 2: Load GAN, augment data, retrain & compare -- verbose training
# ─────────────────────────────────────────────────────────────────────────────
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.optimizers import Adam

# 1) Reload IQ data & labels
with open("RML2016.10a_dict.pkl","rb") as f:
    raw = pickle.load(f, encoding="latin1")

X, y, snr = [], [], []
for (mod, S), sigs in raw.items():
    for sig in sigs:
        X.append(np.vstack([sig[0], sig[1]]).T)
        y.append(mod)
        snr.append(S)
X   = np.array(X, dtype=np.float32)
y   = np.array(y)
snr = np.array(snr)

# 2) Encode and split
le            = LabelEncoder(); y_enc = le.fit_transform(y)
mask          = snr > 6
X_train_real  = X[mask]; y_train_real = y_enc[mask]
X_test, y_test= X, y_enc
snr_test      = snr
n_classes     = len(le.classes_)

# 3) Load pre-trained GAN generator
gen = load_model("saved_models/gan_generator.keras")

# 4) Synthesize same-size dataset
latent_dim = gen.input[0].shape[-1]
n_syn      = len(X_train_real)
z, lbls    = np.random.randn(n_syn, latent_dim), np.random.randint(0, n_classes, n_syn)
X_syn      = gen.predict([z, lbls], verbose=1)
y_syn      = lbls

# 5) Augment & shuffle
X_aug = np.vstack([X_train_real, X_syn])
y_aug = np.concatenate([y_train_real, y_syn])
perm  = np.random.permutation(len(X_aug))
X_aug, y_aug = X_aug[perm], y_aug[perm]

# 6) Classifier definition
def make_clf(input_shape, n_classes):
    m = Sequential([
        LSTM(128, return_sequences=True, input_shape=input_shape),
        Dropout(0.5),
        LSTM(128, return_sequences=False),
        Dropout(0.2),
        Dense(128, activation='relu'),
        Dropout(0.1),
        Dense(n_classes, activation='softmax')
    ])
    m.compile(optimizer=Adam(1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    return m

# 7) Baseline (real-only)
baseline = make_clf((128,2), n_classes)
baseline.fit( X_train_real, y_train_real,
              epochs=50, batch_size=64, verbose=1 )
y_base = baseline.predict(X_test, verbose=1).argmax(axis=1)
acc_base = accuracy_score(y_test, y_base)

# 8) GAN-augmented
aug = make_clf((128,2), n_classes)
aug.fit( X_aug, y_aug,
         epochs=50, batch_size=64, verbose=1 )
y_aug = aug.predict(X_test, verbose=1).argmax(axis=1)
acc_aug = accuracy_score(y_test, y_aug)

print(f"\nBaseline acc: {acc_base:.4f}    Augmented acc: {acc_aug:.4f}")

# 9) Accuracy vs SNR plot
snrs = np.unique(snr_test)
acc_base_snr = [accuracy_score(y_test[snr_test==s], y_base[snr_test==s]) for s in snrs]
acc_aug_snr  = [accuracy_score(y_test[snr_test==s], y_aug[snr_test==s])  for s in snrs]

plt.figure(figsize=(8,5))
plt.plot(snrs, acc_base_snr, label='Baseline')
plt.plot(snrs, acc_aug_snr,  label='Augmented')
plt.xlabel("SNR [dB]"); plt.ylabel("Accuracy")
plt.title("Accuracy vs SNR")
plt.legend(); plt.grid(True); plt.tight_layout(); plt.show()

# 10) Confusion matrices
cm_b = confusion_matrix(y_test, y_base)
cm_a = confusion_matrix(y_test, y_aug)
fig, ax = plt.subplots(1,2,figsize=(14,6))
sns.heatmap(cm_b, annot=True, fmt="d", cmap="Blues",
            xticklabels=le.classes_, yticklabels=le.classes_, ax=ax[0])
ax[0].set_title("Baseline CM")
sns.heatmap(cm_a, annot=True, fmt="d", cmap="Blues",
            xticklabels=le.classes_, yticklabels=le.classes_, ax=ax[1])
ax[1].set_title("Augmented CM")
for a in ax: a.set_xlabel("Pred"); a.set_ylabel("True")
plt.tight_layout(); plt.show()


In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# Cell: Compare Original 2024 LSTM vs Baseline vs GAN‐Augmented
# ─────────────────────────────────────────────────────────────────────────────
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix
from tensorflow.keras.models import load_model

# Assuming X_test, y_test, snr_test, le, baseline_clf, aug_clf exist in namespace
# If not, reimport/define/preload them as in previous cells.

# 1) Load original 2024 model
orig_clf = load_model("lstm_rnn_2024.keras")

# 2) Predict and evaluate
y_orig_pred = orig_clf.predict(X_test, verbose=0).argmax(axis=1)
acc_orig     = accuracy_score(y_test, y_orig_pred)
y_base_pred  = baseline_clf.predict(X_test, verbose=0).argmax(axis=1)
acc_base     = accuracy_score(y_test, y_base_pred)
y_aug_pred   = aug_clf.predict(X_test, verbose=0).argmax(axis=1)
acc_aug      = accuracy_score(y_test, y_aug_pred)

print(f"Original 2024 model accuracy: {acc_orig:.4f}")
print(f"Baseline (real‐only) accuracy: {acc_base:.4f}")
print(f"GAN‐Augmented accuracy:      {acc_aug:.4f}")

# 3) Accuracy vs SNR for all three
snr_vals = np.unique(snr_test)
acc_orig_snr = [accuracy_score(y_test[snr_test==s], y_orig_pred[snr_test==s]) for s in snr_vals]
acc_base_snr = [accuracy_score(y_test[snr_test==s], y_base_pred[snr_test==s]) for s in snr_vals]
acc_aug_snr  = [accuracy_score(y_test[snr_test==s], y_aug_pred[snr_test==s])  for s in snr_vals]

plt.figure(figsize=(8,5))
plt.plot(snr_vals, acc_orig_snr, label='Original 2024')
plt.plot(snr_vals, acc_base_snr, label='Baseline')
plt.plot(snr_vals, acc_aug_snr,  label='Augmented')
plt.xlabel("SNR [dB]"); plt.ylabel("Accuracy")
plt.title("Accuracy vs SNR: Original vs Baseline vs Augmented")
plt.legend(); plt.grid(True); plt.tight_layout(); plt.show()

# 4) Confusion matrices side by side
cm_orig = confusion_matrix(y_test, y_orig_pred)
cm_base = confusion_matrix(y_test, y_base_pred)
cm_aug  = confusion_matrix(y_test, y_aug_pred)

fig, axs = plt.subplots(1, 3, figsize=(18,6))
for ax, cm, title in zip(
    axs,
    [cm_orig, cm_base, cm_aug],
    ["Original 2024", "Baseline", "GAN‐Augmented"]
):
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                xticklabels=le.classes_, yticklabels=le.classes_, ax=ax)
    ax.set_title(title)
    ax.set_xlabel("Predicted"); ax.set_ylabel("True")

plt.tight_layout(); plt.show()
